## PyBDSF - LoTSS DR1 COMPACT V2

***

This notebook outputs a table with the association between the raw PyBDSF catalogue and the final optical catalogue. 
It includes:
* The [original catalogues](#Original-catalogues) 
* The [cleaned catalogues](#Cleaned-catalogues)
* The [ouput table](#Output)

The output table consists of:
* `pybdsf_name` : original raw catalogue PyBDSF ` Source_Name `
* `association_name` : optical v1.2 catalogue `Source_Name`
* Diagnosis `flag` : [artifacts](#Artifacts) (1), [deblended sources](#Deblended-sources) (4), [multi pybdsf component sources](#Multi-pybdsf-component-sources) (2), [single sources](#Single-sources) (8) or a combination of diagnosis [deblended and multi pybdsf component sources](#Deblended-sources-that-were-both-deblended-and-associated-to-a-bigger-source) (6)

***

** Libraries ** 

In [4]:
import pandas as pd
import numpy as np
from astropy.table import Table, join

### Original catalogues

In [5]:
def read_fits(file):
    cat = Table.read(file)
    return cat.to_pandas()

In [6]:
# Raw PyBDSF catalogue
pybdsf = read_fits('data/LOFAR_HBA_T1_DR1_catalog_v0.9.srl.fixed.fits')
# V1.2 LoTSS DR1 catalogues
optical = read_fits('data/v1.2/LOFAR_HBA_T1_DR1_merge_ID_optical_f_v1.2.fits')
components = read_fits('data/v1.2/LOFAR_HBA_T1_DR1_merge_ID_v1.2.comp.fits')
# Artifacts catalogue
artifacts = read_fits('data/LOFAR_HBA_T1_DR1_merge_ID_v1.1.art.fits')

### Cleaned catalogues

 ** Artifact catalogue **
 
Cleaned artifact catalogue with RENAMED source that was picked on 2 different mosaics (and has the same `Source_Name`) and which was classified as an artifact in one mosaic and as a source in other.

In [14]:
# Taking the name of the duplicated entry on the pybdsf catalogue
pybdsf_duplicated = np.array(pybdsf[pybdsf.duplicated('Source_Name')]['Source_Name'])[0]
pybdsf_duplicated

'ILTJ132633.10+484745.7'

In [16]:
# Creating a copy for the new artifact catalogue
artifacts_cleaned = artifacts.copy()
# Replacing the original name
clean_art = artifacts_cleaned['Source_Name'].replace({pybdsf_duplicated:pybdsf_duplicated+'B'})
# Updating the new catalogue
artifacts_cleaned.update(clean_art)

** Pybdsf catalogue **

In [17]:
# Creating a copy for the new pybdsf catalogue
pybdsf_cleaned = pybdsf.copy()
# Replacing the original name
clean_py = pybdsf[pybdsf.duplicated('Source_Name', keep = 'last')].\
        replace({pybdsf_duplicated:pybdsf_duplicated+'B'})
# Updating the new catalogue
pybdsf_cleaned.update(clean_py)

** Components catalogue ** 

Source Names are duplicated in LoTSS DR1 component catalogue (9 equal source name components). This are entries where `Source_Name = Component_Name` (= pybdsf name).

In [18]:
# Eliminating these from the components catalogue (and just keeping the first entry)
components_cleaned = components.drop_duplicates('Component_Name', keep = 'first')

### Creating a diagnosis output table outline 

In [19]:
col_names =  ['pybdsf_name', 'association_name', 'flag']
output_table  = pd.DataFrame(columns = col_names)
output_table

,pybdsf_name,association_name,flag



***
## Artifacts
***


In [20]:
pybdsf_artifacts = pybdsf_cleaned[pybdsf_cleaned['Source_Name'].
                    isin(artifacts_cleaned['Source_Name'])]

Diagnosis artifact table

In [21]:
pybdsf_artifacts_df = pd.DataFrame({
    'pybdsf_name':pybdsf_artifacts['Source_Name'],
    'flag':1,
    'association_name': 'NULL'})

Updating output table

In [22]:
output_table = output_table.append(pybdsf_artifacts_df, ignore_index=True)
print 'Length of output table is', len(output_table), 'and it should be', len(pybdsf_artifacts_df)

Length of output table is 2543 and it should be 2543


***
***
## Deblended sources



Deblended sources are sources that were originally identified by PyBDSF as only one source. In the final optical catalogue (after deblending) they can have:

* 2 (or more) different `Source_Name`
* 1 has `Source_Name` equal to PyBDSF and one (or more) different **

** PyBDSF deblended sources should have 2 different Source_Names after deblending, however in some cases, one of the deblended sources can keep the original PyBDSF name while the other change its source name. For that reason some PyBDSF sources should appear on the optical catalogue and others should not.

***

Having a look at all the sources that were sent to a deblend workflow (`ID_flag = 41`) or selected for deblending on LGZ  (`ID_flag = 42`):

In [23]:
# on the optical catalogue and on the components catalogue
len(optical[(optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)]),\
len(components_cleaned[(components_cleaned['ID_flag'] == 41) | (components_cleaned['ID_flag'] == 42)])

(2435, 3092)

Note that some of them were not deblended (and some are actually part of a bigger source):

In [24]:
len(optical[((optical['ID_flag'] == 41) | (optical['ID_flag'] == 42)) &\
            (optical['Deblended_from'] == '')]),\
len(components_cleaned[((components_cleaned['ID_flag'] == 41) | (components_cleaned['ID_flag'] == 42)) &\
               (components_cleaned['Deblended_from'] == '')]),\
len(components_cleaned[((components_cleaned['ID_flag'] == 41) | (components_cleaned['ID_flag'] == 42)) &\
               (components_cleaned['Deblended_from'] == '')].groupby('Source_Name'))

(641, 646, 641)

Taking the **components catalogue **(which links directly to pybdsf catalogue through `Deblended_from` )

In [25]:
# Total number of pybdsf sources that were deblended
len(components_cleaned[components_cleaned['Deblended_from'] != ''].groupby('Deblended_from'))

935

In [26]:
# Total number of components that make these pybdsf 
components_deblended = components_cleaned[components_cleaned['Deblended_from'] != '']
len(components_deblended)

2446

In [27]:
# Total number of sources after deblending
len(components_deblended.groupby('Source_Name'))

1794

***
** Pybdsf deblended table **
***

We have to search each pybdsf in `components_deblended` and take the correct association name:

In [28]:
deblended_pybdsf_names = components_deblended.drop_duplicates('Deblended_from')['Deblended_from']
len(deblended_pybdsf_names)

935

This is because 1 Pybdsf source can have been deblended into 2 (or more) sources directly; or deblended and then association to other source. For that reason after droping duplicated association names (`Source_Name` below), we are missing some of the original pybdsf names:

In [29]:
len((components_deblended.drop_duplicates('Source_Name')).groupby('Deblended_from'))

908

Creating the deblended table

In [30]:
pybdsf_deblended_df = pd.DataFrame(columns = col_names)
for i in range(0,len(deblended_pybdsf_names)):
    corresp = components_deblended[components_deblended['Deblended_from'] == deblended_pybdsf_names.
                                   iloc[i]][['Deblended_from','Source_Name']]
    pybdsf_deblended = pd.DataFrame(
        {'pybdsf_name':deblended_pybdsf_names.iloc[i],
        'flag':4,
        'association_name': corresp.drop_duplicates()['Source_Name']})
    pybdsf_deblended_df = pybdsf_deblended_df.append(pybdsf_deblended)

Confirming

In [31]:
len(pybdsf_deblended_df),\
len(pybdsf_deblended_df.groupby('pybdsf_name')),\
len(pybdsf_deblended_df.groupby('association_name'))

(1832, 935, 1794)

***
** NOTE **

In [32]:
# We have lost information about 'Component_Name' for this deblended sources
# since a 'Deblended_from' pybdsf source can be made up of several components
# This information can be recovered later if needed:
len(components_deblended[components_deblended['Deblended_from'].\
                                          isin(pybdsf_deblended_df['pybdsf_name'])])

2446

***

This table is no longer a 1-1 relationship:

In [33]:
pybdsf_deblended_df.describe()

,association_name,flag,pybdsf_name
count,1832,1832,1832
unique,1794,1,935
top,ILTJ141420.38+462620.3,4,ILTJ112051.72+474517.1
freq,4,1832,3



***
### Deblended sources that were both deblended and associated to a bigger source
***

Taking the deblended sources that appear more than once on the components catalogue

In [34]:
cond_deblended_big = pd.DataFrame([components_deblended['Source_Name'].value_counts() > 1][0])
len(cond_deblended_big)

1794

In [35]:
deblended_big_index = pd.DataFrame(cond_deblended_big[cond_deblended_big['Source_Name']].index.values)
len(deblended_big_index)

209

In [36]:
deblended_big = pybdsf_deblended_df[pybdsf_deblended_df['association_name'].isin(deblended_big_index[0])]
len(deblended_big)

247

In [37]:
len(pybdsf_deblended_df)

1832

***
** Updating the deblended table **
***

In [38]:
winners = deblended_big.copy()
winners['flag'] = 6

In [39]:
pybdsf_deblended_df.update(winners)

In [40]:
len(pybdsf_deblended_df[pybdsf_deblended_df['flag'] == 6])

247

In [41]:
pybdsf_deblended_df.describe()

,association_name,flag,pybdsf_name
count,1832,1832.0,1832
unique,1794,2.0,935
top,ILTJ141420.38+462620.3,4.0,ILTJ112051.72+474517.1
freq,4,1585.0,3


***
** Updating the output table ** 
***

In [42]:
len(output_table)

2543

In [43]:
output_table = output_table.append(pybdsf_deblended_df,ignore_index=True) 

In [44]:
print 'Table length should be', len(pybdsf_artifacts_df) + len(pybdsf_deblended_df), 'and it is', len(output_table)

Table length should be 4375 and it is 4375


In [45]:
output_table.describe()

,association_name,flag,pybdsf_name
count,4375,4375.0,4375
unique,1795,3.0,3478
top,NULL,1.0,ILTJ145729.01+493344.5
freq,2543,2543.0,3


Checking

In [46]:
len((output_table[output_table['flag']== 6]))

247

In [47]:
len((output_table[output_table['flag']== 6]).groupby('pybdsf_name'))

193

In [48]:
len(output_table[output_table['flag']== 4]) +\
len(output_table[output_table['flag']== 6]) +\
len(output_table[output_table['flag']== 1])

4375

***
** Aditional Calculations**
***
The pybdsf sources that were deblended into several components and after associated have `S_Code`= M on the optical catalogue! 

In [49]:
deblended_optical = optical[optical['Source_Name'].isin(pybdsf_deblended_df['association_name'])]
len(deblended_optical)

1794

Taking the ones with the M code

In [50]:
deblended_associated = deblended_optical[deblended_optical['S_Code']=='M']
len(deblended_associated)

209

In [51]:
deblended_associated[['Source_Name','S_Code', 'Deblended_from']].head() 

,Source_Name,S_Code,Deblended_from
879,ILTJ104717.05+523212.4,M,ILTJ104717.56+523220.2
894,ILTJ104717.96+523226.7,M,ILTJ104717.56+523220.2
5305,ILTJ105235.58+483955.5,M,ILTJ105235.57+483956.3
6302,ILTJ105332.59+472917.3,M,ILTJ105332.98+472916.5
7950,ILTJ105504.32+554817.0,M,ILTJ105504.82+554819.4


In [52]:
deblended_associated_df = pd.DataFrame({
    'pybdsf_name':deblended_associated['Deblended_from'],
    'flag':6,
    'association_name': deblended_associated['Source_Name']})

Confirming that they are the same:

In [53]:
len(deblended_associated[deblended_associated['Source_Name'].
                         isin(pybdsf_deblended_df['association_name'])]),\
len(pybdsf_deblended_df[pybdsf_deblended_df['association_name'].
                        isin(deblended_associated['Source_Name'])]),\
len(pybdsf_deblended_df[pybdsf_deblended_df['association_name'].
                        isin(deblended_associated['Source_Name'])].
                        groupby('association_name'))

(209, 247, 209)

***
## Multi pybdsf component sources

** Sources that were associated (but not have been deblended) - flag 2 **
***

IMPORTANT: Note that S_code (M) that comes from pybdsf cat (and is on comp cat and optical) doesnt mean the sources are an association from several components, while the M code on optical cat that comes from deblended sources means that.

Checking sources that are made up of more than one component

In [54]:
cond_multi = pd.DataFrame([components_cleaned['Source_Name'].value_counts() > 1][0])
len(cond_multi)

318520

Taking the source names

In [55]:
multi_names = pd.DataFrame(cond_multi[cond_multi['Source_Name']].index.values)
len(multi_names)

3774

Taking the components for these source names

In [56]:
multi = components_cleaned[components_cleaned['Source_Name'].isin(multi_names[0])]
len(multi)

9869

Taking just the ones that have multi components but were not deblended

In [57]:
multi_not_deblended = multi[multi['Deblended_from'] == '']

In [58]:
#Number of components
len(multi_not_deblended)

9008

In [59]:
# Which is equal to the number of components = number of pybdsf names (1-1 relation)
len(multi_not_deblended.groupby('Component_Name'))

9008

In [60]:
# This corresponds to the number of sources
len(multi_not_deblended.groupby('Source_Name'))

3565

Confirming the deblended sources + made up of multiple components:

In [61]:
len(multi[multi['Deblended_from'] != ''].groupby('Source_Name')),\
len(multi[multi['Deblended_from'] != ''].groupby('Deblended_from'))

(209, 193)

Creating a multi-component sources table (for just the ones that were not deblended)

In [62]:
multi_not_deblended_df = pd.DataFrame({
    'pybdsf_name':multi_not_deblended['Component_Name'],
    'flag':2,
    'association_name': multi_not_deblended['Source_Name']})

** Updating the output table **

In [63]:
len(output_table)

4375

In [64]:
output_table = output_table.append(multi_not_deblended_df,ignore_index=True) 

In [65]:
print 'Table length should be',\
len(pybdsf_artifacts_df) + len(pybdsf_deblended_df) + len(multi_not_deblended_df),\
'and it is', len(output_table)

Table length should be 13383 and it is 13383


***
## Single sources
***

In [66]:
cond_single = ~cond_multi

In [67]:
cond_single.head()

,Source_Name
ILTJ140313.38+542018.9,False
ILTJ121903.03+471642.8,False
ILTJ113625.39+480303.6,False
ILTJ120454.75+483254.1,False
ILTJ112051.78+474515.2,False


In [68]:
single_names = pd.DataFrame(cond_single[cond_single['Source_Name']].index.values)
print len(single_names)
single_names.head()

314746


,0
0,ILTJ134215.63+561021.1
1,ILTJ105724.12+460748.5
2,ILTJ124928.46+540853.7
3,ILTJ112052.21+521632.6
4,ILTJ150118.88+512152.6


In [69]:
single = components_cleaned[(components_cleaned['Source_Name'].isin(single_names[0])) &\
                           (components_cleaned['Deblended_from'] == '')]
len(single)

313161

In [70]:
len(single.groupby('Component_Name')), len(single.groupby('Source_Name'))

(313161, 313161)

** Single sources table **

In [71]:
single_df = pd.DataFrame({
    'pybdsf_name':single['Component_Name'],
    'flag':8,
    'association_name': single['Source_Name']})

In [72]:
single_df.head()

,association_name,flag,pybdsf_name
3,ILTJ104327.95+521032.6,8,ILTJ104327.95+521032.6
4,ILTJ104330.98+521038.2,8,ILTJ104330.98+521038.2
5,ILTJ104330.98+515535.8,8,ILTJ104330.98+515535.8
6,ILTJ104332.16+520718.7,8,ILTJ104332.16+520718.7
7,ILTJ104332.19+522351.5,8,ILTJ104332.19+522351.5


** Updating the output table **

In [73]:
len(output_table)

13383

In [74]:
output_table = output_table.append(single_df,ignore_index=True) 

In [75]:
print 'Table length should be',\
len(pybdsf_artifacts_df) +\
len(pybdsf_deblended_df) +\
len(multi_not_deblended_df)+\
len(single_df),\
'and it is', len(output_table)

Table length should be 326544 and it is 326544


***
## ARTIFACTS - pybdfs Sources that are not on final cat
***

pybdsf raw catalogue sources that are not on the output table and not anywhere!

all the sources on the output table are on the original raw pybdsf cat

In [76]:
len(output_table[output_table['pybdsf_name'].isin(pybdsf_cleaned['Source_Name'])])

326544

In [77]:
len(output_table)

326544

** But these ones are not anywhere: **

In [78]:
len(pybdsf_cleaned)+1-\
(len(output_table[output_table['flag'] == 1])+\
len(output_table[output_table['flag'] == 2])+\
len(output_table[output_table['flag'] == 8])+\
len(output_table[(output_table['flag'] == 4) | (output_table['flag'] == 6)  ].groupby('pybdsf_name')))

48

In [79]:
len(pybdsf_cleaned) - len(pybdsf_cleaned[pybdsf_cleaned['Source_Name'].isin(output_table['pybdsf_name'])])

48

In [80]:
not_anywhere = pybdsf_cleaned[~pybdsf_cleaned['Source_Name'].isin(output_table['pybdsf_name'])]
print len(not_anywhere)
not_anywhere.head()

48


,Source_Name,RA,E_RA,E_RA_tot,DEC,E_DEC,E_DEC_tot,Peak_flux,E_Peak_flux,E_Peak_flux_tot,...,Maj,E_Maj,Min,E_Min,PA,E_PA,Isl_rms,S_Code,Mosaic_ID,Isl_id
5419,ILTJ120756.73+485018.8,181.986392,4.901848,4.903406,48.838566,4.249418,4.251214,0.099478,0.046243,0.050341,...,26.026527,12.320618,19.785871,9.031758,58.186357,86.705107,0.068662,S,P22Hetde,6913.0
7667,ILTJ111047.25+522332.9,167.696871,0.431311,0.464725,52.392491,0.133896,0.218787,8.078074,0.085039,1.617851,...,58.629697,1.015938,19.633516,0.314406,0.917516,1.461485,0.085039,M,P8Hetdex,3990.0
13591,ILTJ130933.61+484411.7,197.390046,5.363171,5.364254,48.736590,3.359173,3.360901,0.094864,0.035342,0.040112,...,34.838602,13.521077,17.529676,6.268994,65.796654,38.336310,0.057265,S,P35Hetde,4592.0
16146,ILTJ131449.76+471756.2,198.707346,4.620600,4.621857,47.298959,4.092500,4.093918,0.102806,0.036964,0.042298,...,30.311587,10.957390,26.703258,9.549724,76.555696,129.956041,0.066180,S,P35Hetde,7629.0
16156,ILTJ131033.45+463923.3,197.639358,4.780636,4.781851,46.656485,4.192954,4.194339,0.135798,0.038394,0.047029,...,46.391891,13.405002,24.121200,6.672961,50.980399,30.448521,0.080154,S,P35Hetde,7639.0


** 'Not anywhere' artifacts table **

In [82]:
artifacts_not_anywhere_df = pd.DataFrame({
    'pybdsf_name':not_anywhere['Source_Name'],
    'flag':16,
    'association_name': 'NULL'})

In [83]:
artifacts_not_anywhere_df.head()

,association_name,flag,pybdsf_name
5419,NULL,16,ILTJ120756.73+485018.8
7667,NULL,16,ILTJ111047.25+522332.9
13591,NULL,16,ILTJ130933.61+484411.7
16146,NULL,16,ILTJ131449.76+471756.2
16156,NULL,16,ILTJ131033.45+463923.3


** Updating the output table **

In [84]:
output_table = output_table.append(artifacts_not_anywhere_df,ignore_index=True) 

In [85]:
len(output_table)

326592

In [87]:
print 'Table length should be',\
len(pybdsf_artifacts_df) +\
len(pybdsf_deblended_df) +\
len(multi_not_deblended_df)+\
len(single_df)+\
len(artifacts_not_anywhere_df),\
'and it is', len(output_table)

Table length should be 326592 and it is 326592


## Q2

Sources where the component got the same name the pybdsf (= deblended from)

Some of these sources seem that were deblended - because they have the deblended from, but pybdsf name = component name = source name, just appear once on the component cat (eg below)

In [637]:
bb = components[components['Deblended_from'] == components['Component_Name']]

In [638]:
len(bb)

40

In [1077]:
bb[['Source_Name', 'Component_Name']].head()

,Source_Name,Component_Name
9120,ILTJ105551.87+543437.5,ILTJ105552.05+543435.4
25451,ILTJ110953.70+482721.1,ILTJ110953.70+482721.1
31806,ILTJ111516.43+531910.3,ILTJ111516.63+531909.6
31851,ILTJ111519.16+531849.7,ILTJ111519.16+531844.2
34760,ILTJ111744.40+484605.9,ILTJ111744.40+484605.9


In [921]:
len(bb.groupby('Deblended_from'))

40

In [1114]:
aa = output_table[output_table['pybdsf_name'].isin(bb['Component_Name'])]
aa.head()

,association_name,flag,pybdsf_name
2587,ILTJ105551.87+543437.5,6,ILTJ105552.05+543435.4
2662,ILTJ110953.70+482721.1,4,ILTJ110953.70+482721.1
2685,ILTJ111516.43+531910.3,6,ILTJ111516.63+531909.6
2686,ILTJ111519.16+531849.7,6,ILTJ111519.16+531844.2
2705,ILTJ111744.40+484605.9,4,ILTJ111744.40+484605.9


The ones with flag 6 might have been associated and then deblended

In [1171]:
aa[(aa['flag'] == 6) & (aa['association_name'] == aa['pybdsf_name']) ]

,association_name,flag,pybdsf_name
4025,ILTJ140754.04+482535.3,6,ILTJ140754.04+482535.3


In [1170]:
components_cleaned[components_cleaned['Deblended_from'] == 'ILTJ140753.56+482535.4']

,Source_Name,RA,E_RA,DEC,E_DEC,Peak_flux,E_Peak_flux,Total_flux,E_Total_flux,Maj,...,Isl_rms,S_Code,Mosaic_ID,Number_Masked,Number_Pointings,Masked_Fraction,Ng,ID_flag,Component_Name,Deblended_from
266949,ILTJ140753.12+482535.5,211.971343,0.476027,48.426537,0.402119,0.371978,0.055624,0.484848,0.088701,7.278937,...,0.052889,S,P211+50,1,1,1.0,1,42,ILTJ140753.12+482535.5,ILTJ140753.56+482535.4
266962,ILTJ140754.04+482535.3,211.975220,0.478939,48.426491,0.428066,0.350956,0.054989,0.432479,0.089364,7.239739,...,0.052889,S,P211+50,1,1,1.0,1,42,ILTJ140754.05+482535.3,ILTJ140753.56+482535.4


In [1169]:
components_cleaned[components_cleaned['Source_Name'] == 'ILTJ140754.04+482535.3']

,Source_Name,RA,E_RA,DEC,E_DEC,Peak_flux,E_Peak_flux,Total_flux,E_Total_flux,Maj,...,Isl_rms,S_Code,Mosaic_ID,Number_Masked,Number_Pointings,Masked_Fraction,Ng,ID_flag,Component_Name,Deblended_from
266960,ILTJ140754.04+482535.3,211.975151,0.647060,48.426495,0.418631,0.351744,0.058656,0.519084,0.090599,8.310386,...,0.055421,S,P213+47,1,1,1.0,1,42,ILTJ140754.04+482535.3,ILTJ140754.04+482535.3
266962,ILTJ140754.04+482535.3,211.975220,0.478939,48.426491,0.428066,0.350956,0.054989,0.432479,0.089364,7.239739,...,0.052889,S,P211+50,1,1,1.0,1,42,ILTJ140754.05+482535.3,ILTJ140753.56+482535.4


** BUT These ones got the deblended flag AND they dont seem to have been deblended **

In [1159]:
cc = aa[aa['flag'] == 4]
cc['pybdsf_name'].isin(artifacts_cleaned['Source_Name'])

2662    False
2705    False
2913    False
3376    False
3377    False
3512    False
3628    False
3629    False
4073    False
4074    False
Name: pybdsf_name, dtype: bool

In [1138]:
output_table[output_table['pybdsf_name'] == 'ILTJ141528.59+490824.6']

,association_name,flag,pybdsf_name
4073,ILTJ141528.59+490824.6,4,ILTJ141528.59+490824.6


In [1139]:
components_cleaned[components_cleaned['Deblended_from'] == 'ILTJ141528.59+490824.6']

,Source_Name,RA,E_RA,DEC,E_DEC,Peak_flux,E_Peak_flux,Total_flux,E_Total_flux,Maj,...,Isl_rms,S_Code,Mosaic_ID,Number_Masked,Number_Pointings,Masked_Fraction,Ng,ID_flag,Component_Name,Deblended_from
272909,ILTJ141528.59+490824.6,213.86913,3.44242,49.140168,3.327811,0.215568,0.057101,4.319515,0.059889,36.922007,...,0.098597,S,P213+47,1,1,1.0,1,42,ILTJ141528.59+490824.6,ILTJ141528.59+490824.6


In [1140]:
components_cleaned[components_cleaned['Source_Name'] == 'ILTJ141528.59+490824.6']

,Source_Name,RA,E_RA,DEC,E_DEC,Peak_flux,E_Peak_flux,Total_flux,E_Total_flux,Maj,...,Isl_rms,S_Code,Mosaic_ID,Number_Masked,Number_Pointings,Masked_Fraction,Ng,ID_flag,Component_Name,Deblended_from
272909,ILTJ141528.59+490824.6,213.86913,3.44242,49.140168,3.327811,0.215568,0.057101,4.319515,0.059889,36.922007,...,0.098597,S,P213+47,1,1,1.0,1,42,ILTJ141528.59+490824.6,ILTJ141528.59+490824.6


In [1165]:
optical[optical['Source_Name'] == 'ILTJ140754.04+482535.3']['objID']

262216    1.661121e+17
Name: objID, dtype: float64

In [1172]:
len(components_cleaned[components_cleaned['Source_Name'] == components_cleaned['Component_Name']])

314737

***

## Output

In [1068]:
output_table.describe() 

,association_name,flag,pybdsf_name
count,326544,326544.0,326544
unique,318521,5.0,325646
top,NULL,8.0,ILTJ144319.75+542030.3
freq,2543,313161.0,3


In [1069]:
output_table.head()

,association_name,flag,pybdsf_name
0,NULL,1,ILTJ121434.78+503002.8
1,NULL,1,ILTJ121434.25+503055.7
2,NULL,1,ILTJ121433.20+503012.3
3,NULL,1,ILTJ121258.75+484808.1
4,NULL,1,ILTJ121257.90+484811.8


In [ ]:
# FLAG THE 10 THAT HAVE NOT BEEN DEBLENDED WITH A SINGLE FLAG (8)
# PUT THAT CALCULATIONS ON THE DEBLENDED PART OF THE NOTEBOOK!